<a href="https://colab.research.google.com/github/Albert2020-05/PINN/blob/main/My.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Part I. Setup Workplace

### Import package and adjust settings

In [25]:
%load_ext autoreload
%autoreload 2
!pip install patchworklib
# conda install --file <requirements.txt>

from __future__ import print_function
from pathlib import Path
import warnings
from tqdm.notebook import tqdm

import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib
from matplotlib import colors
import csv
import os
import random
import itertools

import scipy
from scipy import stats
import sklearn
from sklearn import datasets
from sklearn import svm
from sklearn.svm import SVC, SVR
from sklearn import metrics
from sklearn.metrics import roc_curve, auc, confusion_matrix, accuracy_score, recall_score,f1_score, roc_auc_score
from sklearn.metrics import make_scorer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler #后加
from sklearn.model_selection import RepeatedKFold, GroupShuffleSplit, GridSearchCV
from sklearn.model_selection import cross_validate
from sklearn.linear_model import LogisticRegression, Ridge
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest, chi2, f_classif
from sklearn.neural_network import MLPClassifier

from plotnine import ggplot, aes, geom_point, scale_shape_manual, scale_colour_manual,scale_fill_manual, facet_grid, options, xlab, ylab, ggtitle, geom_hline, scale_x_datetime, theme_bw, facet_wrap, theme
from mizani.breaks import date_breaks
from mizani.formatters import date_format
import patchworklib as pw
#from mpl_toolkits.mplot3d import axes3d
#from mpl_toolkits.mplot3d.art3d import Poly3DCollection
import plotly.express as px

import datetime

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

import sys
#Replace with your working directory
sys.path.append(r'...\Code')

from autarky_module import helper_functions

from sklearn.ensemble import RandomForestRegressor
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
from sklearn.model_selection import ShuffleSplit
from sklearn.metrics import mean_squared_error
from tqdm import tqdm,trange
import random

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
from google.colab import drive
drive.mount('/content/drive')

import os

new_directory = '/content/drive/MyDrive/Colab Notebooks/Predicting microbial water quality in on-site wate/code'  # 指定新的工作目录路径

# 更改当前的工作目录
os.chdir(new_directory)

# 打印当前的工作目录验证更改是否成功
current_directory = os.getcwd()
print("当前工作目录：", current_directory)

output_dir = 'My Output/'
if not os.path.isdir(output_dir): os.mkdir(output_dir) ## create the folder

Mounted at /content/drive
当前工作目录： /content/drive/MyDrive/Colab Notebooks/Predicting microbial water quality in on-site wate/code


### Load data

In [4]:
#Loading the data

#Change path to your own working directory
df_full = pd.read_csv('all_data_final.csv', index_col=0)

#The type to the column 'Time' needs to be changed into a datetime type. Also, for some of the plots that we will create, the 'Time' column should be our index.
df_full['Time']=pd.to_datetime(df_full['Time'])
df_full=df_full.reset_index()
df_full['Time_helper']=df_full['Time']
df_full=df_full.set_index('Time')

#If we want to add additional columns, we can do this here (e.g something like this):
df_full['log ORP']=np.log(df_full['ORP'])
df_full['log FC']=np.log(df_full['Calibrated FC'])

#We can then filter the data if we want to exclude some parts of the data that should not be used for training and testing.
df_full=df_full[((df_full['actual_Reactor']==2) & (df_full['Experiment']=='Standardbetrieb') & (df_full['Time_helper']<np.datetime64('2021-09-01')))|
               ((df_full['actual_Reactor']==1)) |
               ((df_full['actual_Reactor']==2) & (df_full['Experiment']!='Standardbetrieb'))
               ]

#Add two column that classify each datapoint into safe water or unsafe water (a value of 1 means safe, a value of 0 means unsafe.)
df_full['ICC target'] = df_full.apply(helper_functions.assign_ICC_quality, threshold=5, axis=1, )
df_full['MS2 target'] = df_full.apply(helper_functions.assign_MS2_quality, threshold=6, axis=1, )

#Split the data into train and test sets
df_train, df_test = helper_functions.split_data(df_full, include_Std_data_test=True, include_Std_data_train=False)

#Make a dataframe with standard data only
df_StandardBetrieb=df_full[df_full['Experiment']=='Standardbetrieb']

In [ ]:
print(df_train)

                      ID  Reactor  actual_Reactor        FC raw  \
Time                                                              
2021-04-14 14:00:00   49        1               1  5.760000e-09   
2021-04-14 16:00:00   50        1               1  5.850000e-09   
2021-04-14 17:00:00   51        1               1  5.500000e-09   
2021-04-14 18:00:00   52        1               1  5.680000e-09   
2021-04-14 19:00:00   53        1               1  5.530000e-09   
...                  ...      ...             ...           ...   
2021-11-19 09:30:00  714        3               2  2.450000e-10   
2021-11-19 10:00:00  715        3               2  2.240000e-10   
2021-11-19 10:30:00  716        3               2  2.240000e-10   
2021-11-19 11:00:00  717        3               2  2.020000e-10   
2021-11-19 11:30:00  718        3               2  2.150000e-10   

                     Calibrated FC         ORP  Calibrated pH      Turb  \
Time                                                 

In [5]:
label=['MS2 target','LRV MS2']
features=['ORP','Calibrated FC']
output_filename = 'data_groupdr4.xlsx'  # 指定输出文件名 3的尝试是 add the reset 4 not use the dropna().reset_index()
output_path = os.path.join(output_dir, output_filename) # 拼接输出文件的完整路径
df_train = df_train.reset_index()  # 将索引列转换为普通的数据列
# data_train=df_train[[*features, *label, 'Time', 'Experiment', 'Reactor', 'Experiment_replicate']].dropna().reset_index()
data_train=df_train[['Time', *features, *label, 'Experiment', 'Reactor', 'Experiment_replicate']]#.dropna().reset_index()

writer = pd.ExcelWriter(output_path, engine='openpyxl') # 创建一个Excel写入器

grouped = data_train.groupby(['Experiment', 'Reactor', 'Experiment_replicate'])

# for (experiment, reactor, experiment_replicate), group in grouped:

#     print(f"Experiment: {experiment}, Reactor: {reactor}, Experiment_replicate: {experiment_replicate},Rows: {len(group)}")

#     sheet_name = f"{experiment}_{reactor}_{reactor}" # 创建sheet名
#     group.to_excel(writer, sheet_name=sheet_name, index=False) # 将数据存储到对应的sheet中

# writer.save()

In [ ]:
#Save the modified dataset
output_filename = 'output_test.csv'  # 指定输出文件名
output_path = os.path.join(output_dir, output_filename) # 拼接输出文件的完整路径
df_test.to_csv(output_path, index=False) # 将DataFrame保存为CSV文件
output_filename = 'output_train.csv'  # 指定输出文件名
output_path = os.path.join(output_dir, output_filename) # 拼接输出文件的完整路径
df_train.to_csv(output_path, index=False) # 将DataFrame保存为CSV文件
print("文件已保存：", output_path)

# Part II. Model Training

### RF - regression

In [49]:
xls = pd.read_excel('data/data_groupfinal.xlsx', sheet_name=None)
# 迭代处理每个 sheet
dfs = {} #获得最终的dfs
for sheet_name, dfx in xls.items():
    # 计算每行第一列与第一行第一列的差值
    diff = (dfx['Time'] - dfx['Time'].iloc[0]).astype('timedelta64[h]')

    # 将差值添加为新的一列
    dfx['diff'] = diff

    # 打印更新后的 DataFrame
    print(dfx)
    dfs[sheet_name] = dfx

# 打印每个 DataFrame 的内容
for sheet_name, df in dfs.items():
    # N = np.shape(df)[0]
    # print(f"Sheet: {sheet_name}")
    # print(f"N: {N}")
    # print(df)
    # print()
    df = np.array(df)
    # X_train = df[0:5,0:2]
    # print(X_train)
    print(f"Sheet: {sheet_name}")
    N = np.shape(df)[0]
    idx_tr, idx_va, idx_te = (int(N/4*2), int(N/4), int(N/4))
    print(f"x:{idx_tr}, v:{idx_va}, t:{idx_te}")
    X_train = df[0:idx_tr,[1,-1]]
    y_train = df[0:idx_tr,2:3]
    print(X_train)
    print(y_train)

                  Time       ORP  Calibrated FC  MS2 target   LRV MS2  \
0  2021-11-18 07:30:00  754.1970       1.057387         1.0  7.866287   
1  2021-11-18 08:30:00  757.2040       0.874725         1.0  7.866287   
2  2021-11-18 09:30:00  736.8925       0.417142         1.0  7.866287   
3  2021-11-18 10:30:00  707.1045       0.694422         1.0  7.866287   
4  2021-11-18 11:30:00  702.4865       0.702575         1.0  7.866287   
5  2021-11-18 12:30:00  704.9380       0.782522         1.0  7.866287   
6  2021-11-18 13:30:00  700.8350       0.663282         1.0  7.866287   
7  2021-11-18 14:00:00  690.3530       0.563882         1.0  7.866287   
8  2021-11-18 14:30:00  684.3000       0.416201         0.0  3.213075   
9  2021-11-18 15:00:00  667.7835       0.042440         0.0  3.396465   
10 2021-11-18 15:30:00  653.9180       0.035221         0.0  3.565257   
11 2021-11-18 16:00:00  641.8385       0.071800         0.0  5.267862   
12 2021-11-18 16:30:00  639.7345       0.197358    

### Model impletation

In [66]:
# Model impletation
model=RandomForestRegressor(n_estimators=500,oob_score=True,random_state=0,
                                max_features=5#,min_samples_leaf=2
                                )


# scaler = MinMaxScaler() # 创建MinMaxScaler对象
scaler = StandardScaler()

# for (experiment, reactor), group in grouped:
#     pipeline = make_pipeline(scaler, regressor)# 创建一个管道，将标准化和回归器组合在一起
#     N = np.shape(df)[0]
#     idx_tr, idx_va, idx_te = (int(N/4*2), int(N/4), int(N/4))
#     df = np.array(df)
#     X_train = df(0:idx_tr,0:2)
#     y_train = df(0:idx_tr,2:3)
#     pipeline.fit(X_train, y_train) # 使用管道进行训练和预测
#     predictions = pipeline.predict(X_test)

#     pipeline = make_pipeline(scaler, regressor)# 创建一个管道，将标准化和回归器组合在一起
#     X_train = df
#     pipeline.fit(X_train, y_train) # 使用管道进行训练和预测
#     predictions = pipeline.predict(X_test)

#     print(f"Experiment: {experiment}, Reactor: {reactor}, Rows: {len(group)}")

r = []
res = []

for sheet_name, df in dfs.items():
    df = np.array(df)
    X = df[:,[1,-1]]
    y = df[:, 2]
    X = scaler.fit_transform(X)
    y = np.reshape(y, (-1,1))
    y = scaler.fit_transform(y)
    N = np.shape(df)[0]
    idx_tr, idx_va, idx_te = (int(N/4*2), int(N/4*2), int(N/4))
    X_train = X[0:idx_tr, :]
    y_train = y[0:idx_tr, :]
    # 将特征矩阵X进行归一化
    # X_train = scaler.fit_transform(X_train)
    model.fit(X_train, y_train) # 使用管道进行训练和预测
    X_va = X[idx_tr:idx_tr + idx_va, :]
    y_va = y[idx_tr:idx_tr + idx_va, :]
    # X_va = scaler.fit_transform(X_va)
    predictions = model.predict(X_va)
    sub = predictions - y_va
    y_test = y_va
    y_preditc = predictions
    r2 = 1- mean_squared_error(y_test,y_preditc)/ np.var(y_test)
    r.append(sub)
    res.append(predictions)
    print(f"{sheet_name} : {r2}")


Membrane Damage_3_3 : -2.8876062713358173
aeration off_1_1 : 0.24411454106290253
aeration off_2_2 : -0.6272569320546286
chlorine off_1_1 : -978.8807957336318
chlorine off_3_3 : -10.831941275468688
chlorine restart_1_1 : -9.550764218097587
chlorine restart_2_2 : -2.1293456133592614
party_1_1 : 0.6347323787230286
power outage_1_1 : -2.5689731486444014
power outage_3_3 : -0.4313531740772869


In [52]:
# 假设你有实际值和预测值
actual_values = [1, 2, 3, 4, 5]
predicted_values = [8, 2.3, 2.9, 3.8, 4.7]

# 计算 R^2 系数
r2 = r2_score(actual_values, predicted_values)

# 打印 R^2 系数
print("R^2 系数:", r2)

R^2 系数: -3.923


In [ ]:
frame = xl
scaler = StandardScaler() ##
# Model impletation
model=RandomForestRegressor(n_estimators=500,oob_score=True,random_state=0,
                                max_features=5#,min_samples_leaf=2
                                )



for (experiment, reactor), group in grouped:
    pipeline = make_pipeline(scaler, regressor)# 创建一个管道，将标准化和回归器组合在一起
    N = np.shape(df)[0]
    idx_tr, idx_va, idx_te = (int(N/4*2), int(N/4), int(N/4))
    df = np.array(df)
    X_train = df(0:idx_tr,0:2)
    y_train = df(0:idx_tr,2:3)
    pipeline.fit(X_train, y_train) # 使用管道进行训练和预测
    predictions = pipeline.predict(X_test)

    pipeline = make_pipeline(scaler, regressor)# 创建一个管道，将标准化和回归器组合在一起
    X_train = df
    pipeline.fit(X_train, y_train) # 使用管道进行训练和预测
    predictions = pipeline.predict(X_test)

    print(f"Experiment: {experiment}, Reactor: {reactor}, Rows: {len(group)}")

for sheet_name, df in dfs.items():
    pipeline = make_pipeline(scaler, regressor)# 创建一个管道，将标准化和回归器组合在一起
    N = np.shape(df)[0]
    idx_tr, idx_va, idx_te = (int(N/4*2), int(N/4), int(N/4))
    X_train = df[0:idx_tr,0:2]
    y_train = df[0:idx_tr,2:3]
    pipeline.fit(X_train, y_train) # 使用管道进行训练和预测
    X_va = df[idx_tr:idx_va,0:2]
    y_va = df[idx_tr:idx_va,2:3]
    predictions = pipeline.predict(X_va)


random.seed(3)
# validation
model_index=list(frame.index)
val=random.sample(model_index,400)
for j in val:
  model_index.remove(j)


# catagorized
valdata=frame.loc[val,:]
val_x=valdata.iloc[:,4:14]
val_y=valdata.iloc[:,15]
frame=frame.loc[model_index,:]
x_data=frame.iloc[:,4:14]
x_data.index=range(len(x_data))
y_data=frame.iloc[:,15]
y_data.index=range(len(y_data))
y_data = y_data.to_frame()
x_names=x_data.columns.values.tolist()
colnames=y_data.columns.values.tolist()

ss=ShuffleSplit(n_splits=10, test_size=0.1,random_state=0)
#stdsc=StandardScaler()
prelist=[]
vallist=[]
corlist_train=[]
corlist_test=[]
rmsel_train=[]
rmsel_test=[]
o=[]
imp=[]
model=RandomForestRegressor(n_estimators=500,oob_score=True,random_state=0,
                            max_features=5#,min_samples_leaf=2
                            )
    #with tqdm(total=10) as pbar:
for train_index, test_index in ss.split(x_data,y_data):
    x_train=x_data.iloc[train_index,:]
    x_train.columns=x_names

    y_train = y_data.iloc[train_index].copy()

    x_test=x_data.iloc[test_index,:]
    x_test.columns=x_names

    y_test=y_data.iloc[test_index].copy()
    model.fit(x_train,np.array(y_train).ravel())
    val_one=model.predict(val_x)
    vallist.append(val_one.T)
    #pre_one=model.predict(predf)
    #prelist.append(pre_one.T)
    caculate_cor()
    corlist_train.append(r_train[1,0])
    corlist_test.append(r_test[1,0])
    rmsel_train.append(rmse_train)
    rmsel_test.append(rmse_test)
    # scatter_loss_plot()
    o.append(y_train[colnames[0]])
    o.append(y_train_pred[0])
    o.append(y_test[colnames[0]])
    o.append(y_test_pred[0])
    #pbar.update()
    imp.append(model.feature_importances_)

    #plt.show()
cordf=pd.DataFrame({'train':corlist_train,'test':corlist_test,
                    'rmse_train':rmsel_train,'rmse_test':rmsel_test})
obs_pre_df=pd.DataFrame([y_data[colnames[0]],o[1],o[5],o[9],o[13],o[17],o[21],o[25],o[29],o[33],o[37],o[3],o[7],o[11],o[15],o[19],o[23],o[27],o[31],o[35],o[39]]).T
# obs_pre_df=pd.DataFrame([y_data[colnames[0]],]).T
obs_pre_df.columns=(colnames[0],'train1','train2','train3','train4','train5',
                    'train6','train7','train8','train9','train10',
                    'test1','test2','test3','test4','test5',
                    'test6','test7','test8','test9','test10')
    # presult=pd.DataFrame(prelist,columns=['T','C','S','M','L']).T
vresult=pd.DataFrame(vallist,columns=val).T

# print(np.corrcoef(np.array(np.mean(vresult.T)).ravel(),np.array(val_y).ravel())[0,1])
vresult['predict']=np.array(np.mean(vresult.T)).ravel()
vresult['observe']=val_y
vresult['error']=vresult['predict']-vresult['observe']
